<a href="https://colab.research.google.com/github/nicoelbert/vehicleroutingproblem/blob/main/vehicle_routing_solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Vehicle Routing Problem
vehicle routing problem on complete information using simmulated annealing

#Preparation

In [1]:
!git clone 'https://github.com/nicoelbert/vehicleroutingproblem'

Cloning into 'vehicleroutingproblem'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 62 (delta 28), reused 39 (delta 14), pack-reused 0
Unpacking objects: 100% (62/62), done.


In [7]:
#import packages
import pandas as pd

import sys
#import classes


!git pull
%cd vehicleroutingproblem

from classes import classes as cl

Already up to date.
[Errno 2] No such file or directory: 'vehicleroutingproblem'
/content/vehicleroutingproblem


In [3]:
!git pull
!ls

Already up to date.
classes  original_data	README.md  vehicle_routing_solver.ipynb


##Data Import

In [20]:
#depot masterdata

#load Masterdata from csv
url = 'https://raw.githubusercontent.com/nicoelbert/vehicleroutingproblem/main/original_data/Depots.csv'
df_depots = pd.read_csv(url)


#load basetable
url = 'https://raw.githubusercontent.com/nicoelbert/vehicleroutingproblem/main/original_data/preparedJobs.csv'
df_jobs = pd.read_csv(url)


#next steps load data into classes
#create dictionary for each class
df_depots.head()

,Werk,Werk_lon,Werk_lat
0,Neuherberg,10.310178,49.520537
1,Lauterhofen,11.603907,49.361016
2,Altbach,9.377046,48.719103
3,Deisslingen,8.633826,48.126584
4,Bollschweil,7.769376,47.925264


In [35]:
#create and fill dict_depots

dict_depots = {}

for index, row in df_depots.iterrows():
  d = cl.Depot(row['Werk'],row['Werk_lon'],row['Werk_lat'])
  dict_depots[row['Werk']] = d



##Backup

In [36]:
#backup
"""
"Evaluation of Constructionsites"
print(len(df_jobs['plz'].unique()))
print(len(df_jobs['lon'].unique()))
print(len(df_jobs['lat'].unique()))
#print(len(df_jobs[['lon','lat']]()))

df_sites = df_jobs[['lon','lat']].copy()

print(df_sites.shape)

df_sites.drop_duplicates(inplace=True)


print(df_sites.shape)
k.key
"""

'\n"Evaluation of Constructionsites"\nprint(len(df_jobs[\'plz\'].unique()))\nprint(len(df_jobs[\'lon\'].unique()))\nprint(len(df_jobs[\'lat\'].unique()))\n#print(len(df_jobs[[\'lon\',\'lat\']]()))\n\ndf_sites = df_jobs[[\'lon\',\'lat\']].copy()\n\nprint(df_sites.shape)\n\ndf_sites.drop_duplicates(inplace=True)\n\n\nprint(df_sites.shape)\nk.key\n'